In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW22OT-MakeConcordance-SoS.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""
import string

from BibleWordList2 import *

Song01 = [
    ['Song of Solomon','SoS','01','001','1',"The song of songs, which \emph{is} Solomon's."],
    ['Song of Solomon','SoS','01','002','2',"Let him kiss me with the kisses of his mouth: for thy love \emph{is} better than wine."],
    ['Song of Solomon','SoS','01','003','3',"Because of the savour of thy good ointments thy name \emph{is} \emph{as} ointment poured forth, therefore do the virgins love thee."],
    ['Song of Solomon','SoS','01','004','4',"Draw me, we will run after thee: the king hath brought me into his chambers: we will be glad and rejoice in thee, we will remember thy love more than wine: the upright love thee."],
    ['Song of Solomon','SoS','01','005','5',"I \emph{am} black, but comely, O ye daughters of Jerusalem, as the tents of Kedar, as the curtains of Solomon."],
    ['Song of Solomon','SoS','01','006','6',"Look not upon me, because I \emph{am} black, because the sun hath looked upon me: my mother's children were angry with me; they made me the keeper of the vineyards; \emph{but} mine own vineyard have I not kept."],
    ['Song of Solomon','SoS','01','007','7',"Tell me, O thou whom my soul loveth, where thou feedest, where thou makest \emph{thy} \emph{flock} to rest at noon: for why should I be as one that turneth aside by the flocks of thy companions?"],
    ['Song of Solomon','SoS','01','008','8',"If thou know not, O thou fairest among women, go thy way forth by the footsteps of the flock, and feed thy kids beside the shepherds' tents."],
    ['Song of Solomon','SoS','01','009','9',"I have compared thee, O my love, to a company of horses in Pharaoh's chariots."],
    ['Song of Solomon','SoS','01','010','10',"Thy cheeks are comely with rows \emph{of} \emph{jewels}, thy neck with chains \emph{of} \emph{gold}."],
    ['Song of Solomon','SoS','01','011','11',"We will make thee borders of gold with studs of silver."],
    ['Song of Solomon','SoS','01','012','12',"While the king \emph{sitteth} at his table, my spikenard sendeth forth the smell thereof."],
    ['Song of Solomon','SoS','01','013','13',"A bundle of myrrh \emph{is} my wellbeloved unto me; he shall lie all night betwixt my breasts."],
    ['Song of Solomon','SoS','01','014','14',"My beloved \emph{is} unto me \emph{as} a cluster of camphire in the vineyards of En-gedi."],
    ['Song of Solomon','SoS','01','015','15',"Behold, thou \emph{art} fair, my love; behold, thou \emph{art} fair; thou \emph{hast} doves' eyes."],
    ['Song of Solomon','SoS','01','016','16',"Behold, thou \emph{art} fair, my beloved, yea, pleasant: also our bed \emph{is} green."],
    ['Song of Solomon','SoS','01','017','17',"The beams of our house \emph{are} cedar, \emph{and} our rafters of fir."]
]

Song02 = [
['Song of Solomon','SoS','02','001','1',"I \emph{am} the rose of Sharon, \emph{and} the lily of the valleys."],
['Song of Solomon','SoS','02','002','2',"As the lily among thorns, so \emph{is} my love among the daughters."],
['Song of Solomon','SoS','02','003','3',"As the apple tree among the trees of the wood, so \emph{is} my beloved among the sons. I sat down under his shadow with great delight, and his fruit \emph{was} sweet to my taste."],
['Song of Solomon','SoS','02','004','4',"He brought me to the banqueting house, and his banner over me \emph{was} love."],
['Song of Solomon','SoS','02','005','5',"Stay me with flagons, comfort me with apples: for I \emph{am} sick of love."],
['Song of Solomon','SoS','02','006','6',"His left hand \emph{is} under my head, and his right hand doth embrace me."],
['Song of Solomon','SoS','02','007','7',"I charge you, O ye daughters of Jerusalem, by the roes, and by the hinds of the field, that ye stir not up, nor awake \emph{my} love, till he please."],
['Song of Solomon','SoS','02','008','8',"The voice of my beloved! behold, he cometh leaping upon the mountains, skipping upon the hills."],
['Song of Solomon','SoS','02','009','9',"My beloved is like a roe or a young hart: behold, he standeth behind our wall, he looketh forth at the windows, shewing himself through the lattice."],
['Song of Solomon','SoS','02','010','10',"My beloved spake, and said unto me, Rise up, my love, my fair one, and come away."],
['Song of Solomon','SoS','02','011','11',"For, lo, the winter is past, the rain is over \emph{and} gone;"],
['Song of Solomon','SoS','02','012','12',"The flowers appear on the earth; the time of the singing \emph{of} \emph{birds} is come, and the voice of the turtle is heard in our land;"],
['Song of Solomon','SoS','02','013','13',"The fig tree putteth forth her green figs, and the vines \emph{with} the tender grape give a \emph{good} smell. Arise, my love, my fair one, and come away."],
['Song of Solomon','SoS','02','014','14',"O my dove, \emph{that} \emph{art} in the clefts of the rock, in the secret \emph{places} of the stairs, let me see thy countenance, let me hear thy voice; for sweet \emph{is} thy voice, and thy countenance \emph{is} comely."],
['Song of Solomon','SoS','02','015','15',"Take us the foxes, the little foxes, that spoil the vines: for our vines \emph{have} tender grapes."],
['Song of Solomon','SoS','02','016','16',"My beloved \emph{is} mine, and I \emph{am} his: he feedeth among the lilies."],
['Song of Solomon','SoS','02','017','17',"Until the day break, and the shadows flee away, turn, my beloved, and be thou like a roe or a young hart upon the mountains of Bether."],
]

Song03 = [
['Song of Solomon','SoS','03','001','1',"By night on my bed I sought him whom my soul loveth: I sought him, but I found him not."],
['Song of Solomon','SoS','03','002','2',"I will rise now, and go about the city in the streets, and in the broad ways I will seek him whom my soul loveth: I sought him, but I found him not."],
['Song of Solomon','SoS','03','003','3',"The watchmen that go about the city found me: \emph{to} \emph{whom} \emph{I} \emph{said}, Saw ye him whom my soul loveth?"],
['Song of Solomon','SoS','03','004','4',"\emph{It} \emph{was} but a little that I passed from them, but I found him whom my soul loveth: I held him, and would not let him go, until I had brought him into my mother's house, and into the chamber of her that conceived me."],
['Song of Solomon','SoS','03','005','5',"I charge you, O ye daughters of Jerusalem, by the roes, and by the hinds of the field, that ye stir not up, nor awake \emph{my} love, till he please."],
['Song of Solomon','SoS','03','006','6',"Who \emph{is} this that cometh out of the wilderness like pillars of smoke, perfumed with myrrh and frankincense, with all powders of the merchant?"],
['Song of Solomon','SoS','03','007','7',"Behold his bed, which \emph{is} Solomon's; threescore valiant men \emph{are} about it, of the valiant of Israel."],
['Song of Solomon','SoS','03','008','8',"They all hold swords, \emph{being} expert in war: every man \emph{hath} his sword upon his thigh because of fear in the night."],
['Song of Solomon','SoS','03','009','9',"King Solomon made himself a chariot of the wood of Lebanon."],
['Song of Solomon','SoS','03','010','10',"He made the pillars thereof \emph{of} silver, the bottom thereof \emph{of} gold, the covering of it \emph{of} purple, the midst thereof being paved \emph{with} love, for the daughters of Jerusalem."],
['Song of Solomon','SoS','03','011','11',"Go forth, O ye daughters of Zion, and behold king Solomon with the crown wherewith his mother crowned him in the day of his espousals, and in the day of the gladness of his heart."]
]

Song04 = [
['Song of Solomon','SoS','04','001','1',"Behold, thou \emph{art} fair, my love; behold, thou \emph{art} fair; thou \emph{hast} doves' eyes within thy locks: thy hair \emph{is} as a flock of goats, that appear from mount Gilead."],
['Song of Solomon','SoS','04','002','2',"Thy teeth \emph{are} like a flock \emph{of} \emph{sheep} \emph{that} \emph{are} \emph{even} shorn, which came up from the washing; whereof every one bear twins, and none \emph{is} barren among them."],
['Song of Solomon','SoS','04','003','3',"Thy lips \emph{are} like a thread of scarlet, and thy speech \emph{is} comely: thy temples \emph{are} like a piece of a pomegranate within thy locks."],
['Song of Solomon','SoS','04','004','4',"Thy neck \emph{is} like the tower of David builded for an armoury, whereon there hang a thousand bucklers, all shields of mighty men."],
['Song of Solomon','SoS','04','005','5',"Thy two breasts \emph{are} like two young roes that are twins, which feed among the lilies."],
['Song of Solomon','SoS','04','006','6',"Until the day break, and the shadows flee away, I will get me to the mountain of myrrh, and to the hill of frankincense."],
['Song of Solomon','SoS','04','007','7',"Thou \emph{art} all fair, my love; \emph{there} \emph{is} no spot in thee."],
['Song of Solomon','SoS','04','008','8',"Come with me from Lebanon, \emph{my} spouse, with me from Lebanon: look from the top of Amana, from the top of Shenir and Hermon, from the lions' dens, from the mountains of the leopards."],
['Song of Solomon','SoS','04','009','9',"Thou hast ravished my heart, my sister, \emph{my} spouse; thou hast ravished my heart with one of thine eyes, with one chain of thy neck."],
['Song of Solomon','SoS','04','010','10',"How fair is thy love, my sister, \emph{my} spouse! how much better is thy love than wine! and the smell of thine ointments than all spices!"],
['Song of Solomon','SoS','04','011','11',"Thy lips, O \emph{my} spouse, drop \emph{as} the honeycomb: honey and milk \emph{are} under thy tongue; and the smell of thy garments \emph{is} like the smell of Lebanon."],
['Song of Solomon','SoS','04','012','12',"A garden inclosed \emph{is} my sister, \emph{my} spouse; a spring shut up, a fountain sealed."],
['Song of Solomon','SoS','04','013','13',"Thy plants \emph{are} an orchard of pomegranates, with pleasant fruits; camphire, with spikenard,"],
['Song of Solomon','SoS','04','014','14',"Spikenard and saffron; calamus and cinnamon, with all trees of frankincense; myrrh and aloes, with all the chief spices:"],
['Song of Solomon','SoS','04','015','15',"A fountain of gardens, a well of living waters, and streams from Lebanon."],
['Song of Solomon','SoS','04','016','16',"Awake, O north wind; and come, thou south; blow upon my garden, \emph{that} the spices thereof may flow out. Let my beloved come into his garden, and eat his pleasant fruits."]
]

Song05 = [
['Song of Solomon','SoS','05','001','1',"I am come into my garden, my sister, \emph{my} spouse: I have gathered my myrrh with my spice; I have eaten my honeycomb with my honey; I have drunk my wine with my milk: eat, O friends; drink, yea, drink abundantly, O beloved."],
['Song of Solomon','SoS','05','002','2',"I sleep, but my heart waketh: \emph{it} \emph{is} the voice of my beloved that knocketh, \emph{saying}, Open to me, my sister, my love, my dove, my undefiled: for my head is filled with dew, \emph{and} my locks with the drops of the night."],
['Song of Solomon','SoS','05','003','3',"I have put off my coat; how shall I put it on? I have washed my feet; how shall I defile them?"],
['Song of Solomon','SoS','05','004','4',"My beloved put in his hand by the hole \emph{of} \emph{the} \emph{door}, and my bowels were moved for him."],
['Song of Solomon','SoS','05','005','5',"I rose up to open to my beloved; and my hands dropped \emph{with} myrrh, and my fingers \emph{with} sweet smelling myrrh, upon the handles of the lock."],
['Song of Solomon','SoS','05','006','6',"I opened to my beloved; but my beloved had withdrawn himself, \emph{and} was gone: my soul failed when he spake: I sought him, but I could not find him; I called him, but he gave me no answer."],
['Song of Solomon','SoS','05','007','7',"The watchmen that went about the city found me, they smote me, they wounded me; the keepers of the walls took away my veil from me."],
['Song of Solomon','SoS','05','008','8',"I charge you, O daughters of Jerusalem, if ye find my beloved, that ye tell him, that I \emph{am} sick of love."],
['Song of Solomon','SoS','05','009','9',"What \emph{is} thy beloved more than \emph{another} beloved, O thou fairest among women? what \emph{is} thy beloved more than \emph{another} beloved, that thou dost so charge us?"],
['Song of Solomon','SoS','05','010','10',"My beloved \emph{is} white and ruddy, the chiefest among ten thousand."],
['Song of Solomon','SoS','05','011','11',"His head \emph{is} \emph{as} the most fine gold, his locks \emph{are} bushy, \emph{and} black as a raven."],
['Song of Solomon','SoS','05','012','12',"His eyes \emph{are} as \emph{the} \emph{eyes} of doves by the rivers of waters, washed with milk, \emph{and} fitly set."],
['Song of Solomon','SoS','05','013','13',"His cheeks \emph{are} as a bed of spices, \emph{as} sweet flowers: his lips \emph{like} lilies, dropping sweet smelling myrrh."],
['Song of Solomon','SoS','05','014','14',"His hands \emph{are} \emph{as} gold rings set with the beryl: his belly \emph{is} \emph{as} bright ivory overlaid \emph{with} sapphires."],
['Song of Solomon','SoS','05','015','15',"His legs \emph{are} \emph{as} pillars of marble, set upon sockets of fine gold: his countenance \emph{is} as Lebanon, excellent as the cedars."],
['Song of Solomon','SoS','05','016','16',"His mouth \emph{is} most sweet: yea, he \emph{is} altogether lovely. This \emph{is} my beloved, and this \emph{is} my friend, O daughters of Jerusalem."]
]

Song06 = [
['Song of Solomon','SoS','06','001','1',"Whither is thy beloved gone, O thou fairest among women? whither is thy beloved turned aside? that we may seek him with thee."],
['Song of Solomon','SoS','06','002','2',"My beloved is gone down into his garden, to the beds of spices, to feed in the gardens, and to gather lilies."],
['Song of Solomon','SoS','06','003','3',"I \emph{am} my beloved's, and my beloved \emph{is} mine: he feedeth among the lilies."],
['Song of Solomon','SoS','06','004','4',"Thou \emph{art} beautiful, O my love, as Tirzah, comely as Jerusalem, terrible as \emph{an} \emph{army} with banners."],
['Song of Solomon','SoS','06','005','5',"Turn away thine eyes from me, for they have overcome me: thy hair \emph{is} as a flock of goats that appear from Gilead."],
['Song of Solomon','SoS','06','006','6',"Thy teeth \emph{are} as a flock of sheep which go up from the washing, whereof every one beareth twins, and \emph{there} \emph{is} not one barren among them."],
['Song of Solomon','SoS','06','007','7',"As a piece of a pomegranate \emph{are} thy temples within thy locks."],
['Song of Solomon','SoS','06','008','8',"There are threescore queens, and fourscore concubines, and virgins without number."],
['Song of Solomon','SoS','06','009','9',"My dove, my undefiled is \emph{but} one; she \emph{is} the \emph{only} one of her mother, she \emph{is} the choice \emph{one} of her that bare her. The daughters saw her, and blessed her; \emph{yea}, the queens and the concubines, and they praised her."],
['Song of Solomon','SoS','06','010','10',"Who \emph{is} she \emph{that} looketh forth as the morning, fair as the moon, clear as the sun, \emph{and} terrible as \emph{an} \emph{army} with banners?"],
['Song of Solomon','SoS','06','011','11',"I went down into the garden of nuts to see the fruits of the valley, \emph{and} to see whether the vine flourished, \emph{and} the pomegranates budded."],
['Song of Solomon','SoS','06','012','12',"Or ever I was aware, my soul made me \emph{like} the chariots of Amminadib."],
['Song of Solomon','SoS','06','013','13',"Return, return, O Shulamite; return, return, that we may look upon thee. What will ye see in the Shulamite? As it were the company of two armies."]
]

Song07 = [
    ['Song of Solomon','SoS','07','001','1',"How beautiful are thy feet with shoes, O prince's daughter! the joints of thy thighs \emph{are} like jewels, the work of the hands of a cunning workman."],
    ['Song of Solomon','SoS','07','002','2',"Thy navel \emph{is} \emph{like} a round goblet, \emph{which} wanteth not liquor: thy belly \emph{is} \emph{like} an heap of wheat set about with lilies."],
    ['Song of Solomon','SoS','07','003','3',"Thy two breasts \emph{are} like two young roes \emph{that} \emph{are} twins."],
    ['Song of Solomon','SoS','07','004','4',"Thy neck \emph{is} as a tower of ivory; thine eyes \emph{like} the fishpools in Heshbon, by the gate of Bath-rabbim: thy nose \emph{is} as the tower of Lebanon which looketh toward Damascus."],
    ['Song of Solomon','SoS','07','005','5',"Thine head upon thee \emph{is} like Carmel, and the hair of thine head like purple; the king \emph{is} held in the galleries."],
    ['Song of Solomon','SoS','07','006','6',"How fair and how pleasant art thou, O love, for delights!"],
    ['Song of Solomon','SoS','07','007','7',"This thy stature is like to a palm tree, and thy breasts to clusters \emph{of} \emph{grapes}."],
    ['Song of Solomon','SoS','07','008','8',"I said, I will go up to the palm tree, I will take hold of the boughs thereof: now also thy breasts shall be as clusters of the vine, and the smell of thy nose like apples;"],
    ['Song of Solomon','SoS','07','009','9',"And the roof of thy mouth like the best wine for my beloved, that goeth \emph{down} sweetly, causing the lips of those that are asleep to speak."],
    ['Song of Solomon','SoS','07','010','10',"I \emph{am} my beloved's, and his desire \emph{is} toward me."],
    ['Song of Solomon','SoS','07','011','11',"Come, my beloved, let us go forth into the field; let us lodge in the villages."],
    ['Song of Solomon','SoS','07','012','12',"Let us get up early to the vineyards; let us see if the vine flourish, \emph{whether} the tender grape appear, \emph{and} the pomegranates bud forth: there will I give thee my loves."],
    ['Song of Solomon','SoS','07','013','13',"The mandrakes give a smell, and at our gates \emph{are} all manner of pleasant \emph{fruits}, new and old, \emph{which} I have laid up for thee, O my beloved."]
]

Song08 = [
    ['Song of Solomon','SoS','08','001','1',"O that thou \emph{wert} as my brother, that sucked the breasts of my mother! \emph{when} I should find thee without, I would kiss thee; yea, I should not be despised."],
    ['Song of Solomon','SoS','08','002','2',"I would lead thee, \emph{and} bring thee into my mother's house, \emph{who} would instruct me: I would cause thee to drink of spiced wine of the juice of my pomegranate."],
    ['Song of Solomon','SoS','08','003','3',"His left hand \emph{should} \emph{be} under my head, and his right hand should embrace me."],
    ['Song of Solomon','SoS','08','004','4',"I charge you, O daughters of Jerusalem, that ye stir not up, nor awake \emph{my} love, until he please."],
    ['Song of Solomon','SoS','08','005','5',"Who \emph{is} this that cometh up from the wilderness, leaning upon her beloved? I raised thee up under the apple tree: there thy mother brought thee forth: there she brought thee forth \emph{that} bare thee."],
    ['Song of Solomon','SoS','08','006','6',"Set me as a seal upon thine heart, as a seal upon thine arm: for love \emph{is} strong as death; jealousy \emph{is} cruel as the grave: the coals thereof \emph{are} coals of fire, \emph{which} \emph{hath} \emph{a} most vehement flame."],
    ['Song of Solomon','SoS','08','007','7',"Many waters cannot quench love, neither can the floods drown it: if \emph{a} man would give all the substance of his house for love, it would utterly be contemned."],
    ['Song of Solomon','SoS','08','008','8',"We have a little sister, and she hath no breasts: what shall we do for our sister in the day when she shall be spoken for?"],
    ['Song of Solomon','SoS','08','009','9',"If she \emph{be} a wall, we will build upon her a palace of silver: and if she \emph{be} a door, we will inclose her with boards of cedar."],
    ['Song of Solomon','SoS','08','010','10',"I \emph{am} a wall, and my breasts like towers: then was I in his eyes as one that found favour."],
    ['Song of Solomon','SoS','08','011','11',"Solomon had a vineyard at Baal-hamon; he let out the vineyard unto keepers; every one for the fruit thereof was to bring a thousand \emph{pieces} of silver."],
    ['Song of Solomon','SoS','08','012','12',"My vineyard, which \emph{is} mine, \emph{is} before me: thou, O Solomon, \emph{must} \emph{have} a thousand, and those that keep the fruit thereof two hundred."],
    ['Song of Solomon','SoS','08','013','13',"Thou that dwellest in the gardens, the companions hearken to thy voice: cause me to hear \emph{it}."],
    ['Song of Solomon','SoS','08','014','14',"Make haste, my beloved, and be thou like to a roe or to a young hart upon the mountains of spices."]
]

def main():
    ProcessVerses('Song08',Song08)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{O that thou \emph{wert} as my brother, that sucked the breasts of my mother! \emph{when} I should find thee without, I would kiss thee; yea, I should not be despised.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{I would lead thee, \emph{and} bring thee into my mother's house, \emph{who} would instruct me: I would cause thee to drink of spiced wine of the juice of my pomegranate.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{His left hand \emph{should} \emph{be} under my head, and his right hand should embrace me.}
[4] \textcolor[cmyk]{0.99998,1,0,0}{I charge you, O daughters of Jerusalem, that ye stir not up, nor awake \emph{my} love, until he please.}
[5] \textcolor[cmyk]{0.99998,1,0,0}{Who \emph{is} this that cometh up from the wilderness, leaning upon her beloved? I raised thee up under th

\emph{and} & 1 \\ \hline
\emph{are} & 1 \\ \hline
\emph{a} & 2 \\ \hline
\emph{be} & 3 \\ \hline
\emph{hath} & 1 \\ \hline
\emph{have} & 1 \\ \hline
\emph{is} & 5 \\ \hline
\emph{it} & 1 \\ \hline
\emph{must} & 1 \\ \hline
\emph{my} & 1 \\ \hline
\emph{pieces} & 1 \\ \hline
\emph{should} & 1 \\ \hline
\emph{that} & 1 \\ \hline
\emph{wert} & 1 \\ \hline
\emph{when} & 1 \\ \hline
\emph{which} & 1 \\ \hline
\emph{who} & 1 \\ \hline
a & 12 \\ \hline
all & 1 \\ \hline
and & 6 \\ \hline
apple & 1 \\ \hline
arm & 1 \\ \hline
as & 6 \\ \hline
at & 1 \\ \hline
awake & 1 \\ \hline
bare & 1 \\ \hline
be & 4 \\ \hline
before & 1 \\ \hline
beloved & 2 \\ \hline
boards & 1 \\ \hline
breasts & 3 \\ \hline
bring & 2 \\ \hline
brother & 1 \\ \hline
brought & 2 \\ \hline
build & 1 \\ \hline
can & 1 \\ \hline
cannot & 1 \\ \hline
cause & 2 \\ \hline
cedar & 1 \\ \hline
charge & 1 \\ \hline
coals & 2 \\ \hline
cometh & 1 \\ \hline
companions & 1 \\ \hline
contemned & 1 \\ \hline
cruel & 1 \\ \hline
daught